In [50]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [126]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [205]:
model_name = "gpt-4"
temperature = 0.7
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=4097)

/home/chris/anaconda3/envs/ai-dev/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/chris/anaconda3/envs/ai-dev/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [174]:
# Define your desired data structure.
class Outline(BaseModel):
    intro: str = Field(description="The introduction to the blog post")
    section1: str = Field(description="The first section of the blog post, not including the introduction")
    section2: str = Field(description="The second section of the blog post, not including the introduction")
    # section3: str = Field(description="The third section of the blog post, not including the introduction")
    # section4: str = Field(description="The fourth section of the blog post, not including the introduction")
    # section5: str = Field(description="The fith section of the blog post, not including the introduction")
    conclusion: str = Field(description="The conclusion of the blog post")
    


In [175]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Outline)

In [206]:
outline_human_msg = """Please write a blog post with 2 sections in addition to the introduction and conclusion in the first person about "{input}" 

Each section, including the introduction and conclusion, should be at least 3 paragraphs.

{format_instructions}
"""
input="How to place picture frames on a bookshelf for maximum effect."
prompt = PromptTemplate(
    template=outline_human_msg,
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [207]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"intro": {"title": "Intro", "description": "The introduction to the blog post", "type": "string"}, "section1": {"title": "Section1", "description": "The first section of the blog post, not including the introduction", "type": "string"}, "section2": {"title": "Section2", "description": "The second section of the blog post, not including the introduction", "type": "string"}, "conclusion": {"title": "Conclusion", "description": "The conclusion of the blog post", "type": "string"}}, "required": ["intro", "section1", "section2", "co

In [208]:
_input = prompt.format_prompt(input=input)

In [209]:
output = model(_input.to_string())

In [210]:
print(_input.to_string())

Please write a blog post with 2 sections in addition to the introduction and conclusion in the first person about "How to place picture frames on a bookshelf for maximum effect." 

Each section, including the introduction and conclusion, should be at least 3 paragraphs.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"intro": {"title": "Intro", "description": "The introduction to the blog post", "type": "string"}, "section1": {"title": "Section1", "description": "The first section of the blog post, not including the introduction", "type": "string"}, "section2": {"title": "S

In [211]:
parsed_output = parser.parse(output)

In [212]:
type(parsed_output)

__main__.Outline

In [213]:
parsed_output

Outline(intro="Hello readers, have you ever wondered how to best place picture frames on a bookshelf for maximum effect? Well, you're not alone. Many of us have bookshelves at home that we want to make look attractive, but we're not sure how. In fact, decorating a bookshelf can be a complex task, especially if you want to incorporate picture frames. But don't worry, that's what this blog post is all about. Over the next few paragraphs, I'll share with you some of the best tips and tricks I've learned over the years for placing picture frames on a bookshelf.", section1="To begin with, it's important to understand that balance and symmetry are key when it comes to placing picture frames on a bookshelf. This doesn't mean all picture frames need to be the same size or in the same position. Rather, it means that the overall look of the bookshelf should feel balanced. For instance, you might place a large picture frame on one shelf and then balance it out with a couple of smaller frames on a

In [214]:
section_1_prompt=ChatPromptTemplate.from_template(
    """Below is a short version of a section from a blog post on '{blog_topic}'. 
    Please expand the section by writing at least 5 paragraphs on this topic. 
    Because this is one section in a larger blog post, do not start any paragraph with "Lastly" or "In Conclusion".  
    This secion merely must transition to the next one.  
    The end of this section is NOT the end of the blog post.
   
   
   ### SECTION TOPIC STATEMENT   
   {section_1}
    """
)

section_1_chain = LLMChain(llm=model, prompt=section_1_prompt, output_key="expanded_section_1")


In [215]:
section_2_prompt=ChatPromptTemplate.from_template(
    """Below is a short version of a section from a blog post on '{blog_topic}'. 
    Please expand the section by writing at least 5 paragraphs on this topic.  
    Because this is one section in a larger blog post, do not start any paragraph with "Lastly" or "In Conclusion".  
    This secion merely must transition to the next one.  
    The end of this section is NOT the end of the blog post.
   
   ### SECTION TOPIC STATEMENT   
   {section_2}
    """
)

section_2_chain = LLMChain(llm=model, prompt=section_2_prompt, output_key="expanded_section_2")

In [216]:
overall_chain = SequentialChain(
    chains = [section_1_chain, section_2_chain],
    input_variables = ["blog_topic", "section_1", "section_2"],
    output_variables = [ "expanded_section_1", "expanded_section_2"],
    verbose=True
)

In [217]:
output = overall_chain({
    'section_1': parsed_output.section1,
    'section_2': parsed_output.section2,
    })



> Entering new SequentialChain chain...



> Finished chain.


In [218]:
print(output["expanded_section_1"])

The initial step towards achieving an appealing aesthetic for your bookshelf involves understanding the critical role that balance and symmetry play in placing picture frames. This, however, does not imply that all frames must be identical in size or position. Instead, the premise is that the overall appearance of the bookshelf should exude a certain level of balance. For example, you can strategically place a large picture frame on one shelf, then counterbalance it with a couple of smaller frames placed on a separate shelf. Alternatively, you might opt to position a tall frame on the left side of the shelf and a shorter one on the right, thereby creating an appealing asymmetrical balance. The primary objective is to strike a visually pleasing balance, rather than perfect symmetry, which can sometimes feel rigid and uninteresting.

In addition to balance and symmetry, the use of color in your picture frames can significantly alter the overall aesthetic of your bookshelf. Color can evok

In [219]:
print(output["expanded_section_2"])

Diving deeper into the art of arranging picture frames, we should first discuss the potential of creating a theme. Your bookshelf is more than just a storage space, it's a canvas for showcasing your unique personality and interests. You could choose to display photos from a memorable trip or event, effectively turning your bookshelf into a visual chronicle of your experiences. Alternatively, you might opt for a subject-based theme, such as adorable photos of your pets or heartwarming snaps of your children. By defining a theme, you are giving yourself a more focused direction when it comes to choosing which photos to display, while also adding a personal touch that makes your bookshelf intriguing and engaging.

In addition to theming, the size and shape of your picture frames can be used to relay a narrative. One way to achieve a dynamic look is to mix frames of different sizes. For instance, you could use a large frame for a standout photo that you want to draw attention to, and then 